In [1]:
import yfinance as yf
import yfinance.shared as shared
import time
import pandas as pd
# from datetime import date, timedelta, datetime
from myUtils import pickle_dump, pickle_load, read_symbols_file # NOQA
from myUtils import drop_symbols_all_NaN, chunked_list # NOQA
from myUtils import yf_download_AdjOHLCV_noAutoAdj
from yf_utils import _2_split_train_val_test, _3_random_slices, _4_perf_ranks

verbose = False  # True prints more output

path_dir = "C:/Users/ping/MyDrive/stocks/yfinance/"
path_data_dump = path_dir + "VSCode_dump/"

filename_symbols = path_data_dump + 'vg_symbols_4chars_max.csv'  # symbols text file
filename_pickled_df_OHLCVA_downloaded = 'df_OHLCVA_downloaded '  # OHLCVA downloaded from Yahoo
filename_pickled_df_adjOHLCV = 'df_adjOHLCV'  # adjusted OHLCV
filename_pickled_df_symbols_close = "df_symbols_close"  # symbols' adjusted close
filename_pickled_symbols_df_adjOHLCV =  'symbols_df_adjOHLCV'  # symbols in df_adjOHLCV
filename_pickled_df_c = 'df_close_clean' 

In [3]:
df_c = pickle_load(path_data_dump, filename_pickled_df_c)
df_train, df_val, df_test = _2_split_train_val_test(df_c)
len_df_train = len(df_train)
len_df_val = len(df_val)
len_df_test = len(df_test)

In [4]:
# _n_samples = 2  # number of tuples to create for iloc start_train:end_train:end_eval
_n_samples = 2  # number of tuples to create for iloc start_train:end_train:end_eval
_days_lookbacks = [120, 60, 30]
_days_eval = 10

In [5]:
# create n_samples of iloc start_train:end_train:end_eval using max value in days_loobacks
max_lookback_slices = _3_random_slices(len_df_train, n_samples=_n_samples, days_lookback=max(_days_lookbacks), days_eval=_days_eval)
max_lookback_slices

[(104, 224, 234), (626, 746, 756)]

In [7]:
from yf_utils import _5_lookback_slices
sets_lookback_slices = _5_lookback_slices(max_slices=max_lookback_slices, days_lookbacks=_days_lookbacks, verbose=True)
sets_lookback_slices 

days: 30, (194, 224, 234)
days: 60, (164, 224, 234)
days: 120, (104, 224, 234)

days: 30, (716, 746, 756)
days: 60, (686, 746, 756)
days: 120, (626, 746, 756)



[[(194, 224, 234), (164, 224, 234), (104, 224, 234)],
 [(716, 746, 756), (686, 746, 756), (626, 746, 756)]]

In [ ]:
L = [('grape', 100), ('apple', 15), ('grape', 3), ('apple', 10),
    ('apple', 4), ('banana', 3)]

In [ ]:
def grp_tuples_sort_sum(L, reverse=True):
  # https://stackoverflow.com/questions/2249036/grouping-python-tuple-list
  # https://stackoverflow.com/questions/10695139/sort-a-list-of-tuples-by-2nd-item-integer-value
  """
  Given a list of tuples of (key:value) such as:
  [('grape', 100), ('apple', 15), ('grape', 3), ('apple', 10),
   ('apple', 4), ('banana', 3)]
  Returns list of grouped-sorted-tuples based on summed-values such as:
  [('grape', 103), ('apple', 29), ('banana', 3)] 

  Args:
      L(list of tuples): list of tuples of key(str):value(int) pairs
      reverse(bool): sort order of summed-values of the grouped tuples,
       default is in descending order.  

  Return:
      grp_sorted_list(list of tuples): list of grouped-sorted-tuples
       based on summed-values such as:
       [('grape', 103), ('apple', 29), ('banana', 3)] 
  """ 

  import itertools
  from operator import itemgetter

  grp_list = []
  l = sorted(L)
  it = itertools.groupby(l, itemgetter(0))

  for key, subiter in it:
    # print(f'key: {key}')
    key_sum = sum(item[1] for item in subiter)
    # print(f'key_sum: {key_sum}')
    grp_list.append((key, key_sum))

  grp_sorted_list = sorted(grp_list, key=itemgetter(1), reverse=reverse)

  return grp_sorted_list  


In [ ]:
grp_tuples_sort_sum(L, reverse=True)

In [ ]:
# grp_perf_ranks = {}
grp_most_common_syms = []
# loop thru lists of tuples of start_train:end_train:end_eval, i.e.
#  [[(887, 917, 927), (857, 917, 927), (797, 917, 927)],
#  [(483, 513, 523), (453, 513, 523), (393, 513, 523)]]
for lb_slices in sets_lookback_slices:  
  for lb_slice in lb_slices:  # lb_slice, e.g. (246, 276, 286)
    start_train = lb_slice[0]
    end_train = lb_slice[1]
    start_eval = end_train
    end_eval = lb_slice[2]
    lookback = end_train - start_train
    eval = end_eval - start_eval
    print(f'lb_slices:     {lb_slices}')
    print(f'lb_slice:      {lb_slice}')
    print(f'days lookback: {lookback}')
    print(f'days eval:     {eval}')    
    print(f'start_train:   {start_train}')
    print(f'end_train:     {end_train}')
    # print(f'start_eval:    {start_eval}')
    # print(f'end_eval:      {end_eval}')

    _df = df_train.iloc[start_train:end_train]
    perf_ranks, most_common_syms = _4_perf_ranks(_df, n_top_syms=10)
    # 1 lookback of r_CAGR/UI, r_CAGR/retnStd, r_retnStd/UI
    print(f'perf_ranks: {perf_ranks}')  
    # most common symbols of perf_ranks 
    print(f'most_common_syms: {most_common_syms}')     
    # grp_perf_ranks[lookback] = perf_ranks
    print(f'+++ finish lookback slice {lookback} +++\n')
    grp_most_common_syms.append(most_common_syms)
    
  print(f'grp_most_common_syms: {grp_most_common_syms}')
  print(f'===== finish lookback slices {lb_slices} =====\n')

In [ ]:
# grp_perf_ranks
grp_most_common_syms
# flatten list of lists of (symbols:count)
flat_grp_most_common_syms = [val for sublist in grp_most_common_syms for val in sublist]

sorted_grp_most_common_syms = grp_tuples_sort_sum(flat_grp_most_common_syms, reverse=True)
print(f'len(sorted_grp_most_common_syms): {len(sorted_grp_most_common_syms)}')
sorted_grp_most_common_syms
# top_common_syms = sorted_grp_most_common_syms[0:5]
# top_common_syms = [i[0] for i in top_common_syms]  # get keys from key:value pairs
# top_common_syms 

In [ ]:
top_common_syms = sorted_grp_most_common_syms[0:5]
top_common_syms = [i[0] for i in top_common_syms]  # get keys from key:value pairs
top_common_syms 